In [96]:
!pip install plotly numpy pandas ipykernel nbformat
!pip install plotly numpy pandas ipykernel nbformat


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [97]:
from pyGBM6700e import lab_1
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from copy import deepcopy
import random
import pandas as pd
from plotly.subplots import make_subplots

np.set_printoptions(precision=2)


root_directory = "lab_data"
view_0 = 'PA0'
view_1 = 'LAT'

# Preamble
Please write the RMS calculation function here. You will need this function throughout the assignment, so craft it with care !

In [98]:
def rms(reference:np.ndarray, prediction:np.ndarray) -> float:
    """
    Calculates RMS

    Parameters
    ----------
    reference : np.ndarray
        Reference value
    prediction : np.ndarray
        Calculated/predicted value

    Returns
    -------
    float
        RMS metric
    """
    if reference.shape != prediction.shape:
        raise ValueError('Please provide arrays that are shaped the same ! Reference has shape {refshape} whereas prediction has shape {predshape}'.format(refshape = reference.shape, predshape = prediction.shape))
    rms_value = 0.
    if len(reference.shape) == 2: # Array of shape N, 3:
        rms_value = np.sqrt(np.sum(np.linalg.norm(prediction - reference, axis=1)**2)/(reference.shape[0]))
    else: # Array of shape N, :
        rms_value = np.sqrt(np.sum((prediction - reference)**2)/(reference.shape[0]))
    return rms_value

# Question 1.1
Write the code for finding the over determined system matrix A. Then use the dlt function from lab_1.solver to extract the L parameters vector for each view.

In [99]:
beads_data = lab_1.io.process_files(root_directory)
# ? Filter out non redundant beads
unique_beads = beads_data['2d']['bead'].tolist()
unique_beads = [x.replace('C', 'A') for x in unique_beads]
unique_beads = [x.replace('D', 'B') for x in unique_beads]

# ? Count values
value_count = {}
for k in unique_beads:
    if k in value_count.keys():
        value_count[k] += 1
    else:
        value_count[k] = 1

# ? Exclude the ones that aren't visible from everywhere
for k, v in zip(list(value_count.keys()), list(value_count.values())):
    if v == 1:
        print('Dropped', k)
        del value_count[k]

unique_beads = list(value_count.keys())
unique_beads.extend([x.replace('A', 'C') if 'A' in x else x.replace('B', 'D') for x in unique_beads])
beads_data['2d'] = beads_data['2d'][beads_data['2d']['bead'].isin(unique_beads)]
beads_data['3d'] = beads_data['3d'][beads_data['3d']['bead'].isin(unique_beads)]

Dropped A_4_1
Dropped B_3_5


In [100]:
from pyGBM6700e.lab_1.data import select_view, join_dataframes

def get_a(view:str, data:dict) -> np.ndarray:
    """
    Uses the view name and existing data for pre calculating the system matrix for calculating the 11 DLT coefficients

    Parameters
    ----------
    view : str
        View name
    data : dict
        Data as a dict, with two keys ['2d', '3d']. Each key is associated to a table giving points location in 2d/3d, bead identifier and view name.

    Returns
    -------
    np.ndarray
        A matrix, for the DLT
    """
    A = []
    joint_data = join_dataframes(select_view(data['2d'], view), data['3d'], 'bead')
    # TODO : Using the equations from Appendix 1, write a function to turn pairs of 2D/3D beads data to an overdeterminated system.
    for datum in joint_data.iterrows():
        u_i = datum[1]["x_2d"]
        v_i = datum[1]["y_2d"]
        x_i = datum[1]["x_3d"]
        y_i = datum[1]["y_3d"]
        z_i = datum[1]["z_3d"]
        A.append([x_i, y_i, z_i, 1, 0, 0, 0, 0, -u_i*x_i, -u_i*y_i, -u_i*z_i, -u_i])
        A.append([0, 0, 0, 0, x_i, y_i, z_i, 1, -v_i*x_i, -v_i*y_i, -v_i*z_i, -v_i])
    # HINT 1 : we want to end up with some system AM = 0 where M is a vector containing [L0, L1... L10, 1]. So you just need to return A !
    # HINT 2 : I gave you 'joint_data'. It's a dataframe, so check the columns names !
    return np.array(A)


In [101]:
l_view0 = lab_1.solver.dlt(get_a('Beads2D_'+view_0, beads_data))
l_view1 = lab_1.solver.dlt(get_a('Beads2D_'+view_1, beads_data))

In [102]:
print("DLT parameters for %s : "%view_0, l_view0)
print("DLT parameters for %s : "%view_1, l_view1)

DLT parameters for PA0 :  [ 4.09e-03 -2.57e+00 -2.44e-03 -4.16e-03 -3.60e-02  3.18e-03  2.58e+00
 -5.67e-01  5.50e-04  1.57e-06 -9.44e-06]
DLT parameters for LAT :  [-3.09e+00  1.29e-01 -1.93e-03 -9.36e+02  4.54e-03  4.34e-02  3.10e+00
  1.39e+01 -2.36e-05 -6.61e-04 -1.22e-05]


# Question 1.2

In [103]:
def get_parameters(dlt_params:np.ndarray) -> dict:
    """
    Outputs a dictionnary containing the intrinsic and extrinsic parameters of the system based on the DLT parameters.

    Parameters
    ----------
    dlt_params : np.ndarray
        _description_

    Returns
    -------
    dict
        Dict with keys ['u0', 'v0', 'c_u', 'c_v', 'R', 'source_coordinates']
    """
    out = {}
    l1, l2, l3, l4, l5, l6, l7, l8, l9, l10, l11 = dlt_params
    d = -1/(np.sqrt(l9**2 + l10**2 + l11**2))
    u0 = (l1 * l9 + l2 * l10 + l3 * l11) * d**2
    v0 = (l5 * l9 + l6 * l10 + l7 * l11) * d**2

    cu = np.sqrt(d**2 * ((u0*l9 - l1)**2 + (u0*l10 - l2)**2 + (u0*l11 - l3)**2))
    cv = np.sqrt(d**2 * ((v0*l9 - l5)**2 + (v0*l10 - l6)**2 + (v0*l11 - l7)**2))

    R = []
    r1 = [d/cu *(u0*l9 - l1), d/cu *(u0*l10 - l2), d/cu *(u0*l11 - l3)]
    r2 = [d/cv *(v0*l9 - l5), d/cv *(v0*l10 - l6), d/cv *(v0*l11 - l7)]
    r3 = [l9 * d, l10 * d, l11 * d]
    R.append(r1)
    R.append(r2)
    R.append(r3)

    A = [[l1, l2, l3], [l5, l6, l7], [l9, l10, l11]]
    B = [-l4, -l8, -1]

    A = np.array(A)
    B = np.array(B)
    A_inv = np.linalg.inv(A)
    x,y,z = A_inv.dot(B.T)

    out['u0'] = u0
    out['v0'] = v0
    out['c_u'] = cu
    out['c_v'] = cv
    out['R'] = R
    out['source_coordinates'] = [x,y,z]

    # TODO : Using the equations from Appendix 2, write a function to calculate intrinsic and extrinsic parameters of a system based on the DLT parameters
    return out

print('View %s :'%view_0)
for k, v in get_parameters(l_view0).items():
    print('\t {variable} = {value}'.format(variable = k, value = v))
print('-------------------------------------------------------------')
print('View %s :'%view_1)
for k, v in get_parameters(l_view1).items():
    print('\t {variable} = {value}'.format(variable = k, value = v))

View PA0 :
	 u0 = -5.818320392632722
	 v0 = -145.91496533853473
	 c_u = 4667.8104535235125
	 c_v = 4684.955368831336
	 R = [[0.0028383099743994737, -0.9999955009676524, -0.000970587936685526], [0.01716201640537952, 0.0013246258755954304, 0.9998518443045407], [-0.999848582937606, -0.0028545538049507243, 0.01716574263571952]]
	 source_coordinates = [-1818.7181445750196, -2.8731517662001194, -25.190174003060392]
-------------------------------------------------------------
View LAT :
	 u0 = -29.21576695254617
	 v0 = -152.70327971664815
	 c_u = 4673.68335022721
	 c_v = 4690.201491353493
	 R = [[-0.9993648633679075, 0.0356275622757363, -0.0007393726612318864], [0.0003038194953656494, -0.01852736445514756, 0.9998283074908714], [0.0356077967353628, 0.999194504191617, 0.01850479951043856]]
	 source_coordinates = [-239.20815295327685, 1522.1804797429031, -25.44596310813415]


# Question 1.3

In [104]:
def calculate_3d_point(u0:float, v0:float, u1:float, v1:float, L0:np.ndarray, L1:np.ndarray) -> np.ndarray:
    """
    Calculates a 3D point from 2 2D coordinates pairs and L0 & L1 DLT calibration coefficients

    Parameters
    ----------
    u0 : float
        point 1 x-coordinate
    v0 : float
        point 1 y-coordinate
    u1 : float
        point 2 x-coordinate
    v1 : float
        point 2 y-coordinate
    L0 : np.ndarray
        DLT coefficients for view 1
    L1 : np.ndarray
        DLT coefficients for view 2
    """
    A = []
    # TODO : using Appendix A, write a function that takes 2 pairs of 2D coordinates and 2 11-parameters vectors as an input to return the corresponding 3D point. 
    # HINT 1 : It's the exact same problem as in question 1.1, but now the unknowns are x,y and z !
    # HINT 2 : Refactor the pair of equation in order to have a 4 equations system (2 equations per view, 2 views) for finding a triplet of coordinates : AM = 0 where M = [x, y, z, 1]
    l1, l2, l3, l4, l5, l6, l7, l8, l9, l10, l11 = L0
    A.append([u0*l9 - l1, u0*l10 - l2, u0*l11 - l3, u0-l4])
    A.append([v0*l9 - l5, v0*l10 - l6, v0*l11 - l7, v0-l8])

    l1, l2, l3, l4, l5, l6, l7, l8, l9, l10, l11 = L1
    A.append([u1*l9 - l1, u1*l10 - l2, u1*l11 - l3, u1-l4])
    A.append([v1*l9 - l5, v1*l10 - l6, v1*l11 - l7, v1-l8])
    return lab_1.solver.dlt(np.array(A))

In [105]:
spine = lab_1.structures.Spine(os.path.join(root_directory,'Vertebrae2D.mat'))
colors = px.colors.qualitative.Alphabet
fig = lab_1.render.Plot3D()


for i, v_name in enumerate(spine.vertebrae):
    # TODO : for each vertebra, calculate the location of each anatomical landmark and add its x,y,z coordinates to vert_x, vert_y, vert_z. Each vertebra will then be added to the 3D plot and plotted in the end.
    vert = spine.get(v_name).data # Returns the vertebra data
    """
    HINT : the data is a table containing the landmark name and the x,y coordinates for each view (so 5 columns)
    HINT : Print the variable vert, and add a 'break' instruction to get out of the loop
    For each vertebra, you should have 6 points. Each point is linked to 2 pairs of 2D coordinates, one in each view. These coordinates, with the two 11-DLT parameters will allow you to calculate the 3D position of the 6 anatomical landmarks on the vertebra.
    """
    vert_x, vert_y, vert_z = [], [], []

    for datum in vert.iterrows():
        datum = datum[1]
        u0 = datum["x_Vertebrae_LAT"]
        v0 = datum["y_Vertebrae_LAT"]
        u1 = datum["x_Vertebrae_PA0"]
        v1 = datum["y_Vertebrae_PA0"]
        point = calculate_3d_point(u0, v0, u1, v1, l_view0, l_view1)
        vert_x.append(point[0])
        vert_y.append(point[1])
        vert_z.append(point[2])
    """
    PSEUDOCODE
    for each anatomical_landmark in vertebra_points:
        x_point, y_point, z_point <- calculate_3d_point(anatomical_landmark, l_view0, l_view1)
        Append x_point to vert_x
        Append y_point to vert_y
        Append z_point to vert_z
    """
    fig.scatter_vertebrae(vert_x, vert_y, vert_z, v_name, colors[i]) # Adds the landmarks to the figure
SPINE_3D_POINTS = fig.points # Access all the figure points as an array this way, will be useful in the rest of this assignment
fig.show() # Shows the figure

# Question 2.1

Here I recommend you write a function that samples `n_beads` in the full set, and returns the subset. A code skeleton and some hints can be found below.

In [106]:
def sample_n_beads(original_beads:dict, n_beads:int) -> dict:
    """
    Samples n_beads beads from the original beads set

    Parameters
    ----------
    original_beads : dict
        Dictionnary containing information on 2D and 3D position of the whole set of calibration beads
    n_beads : int
        Number of beads to select

    Returns
    -------
    dict
        Dictionnary containing information on 2D and 3D position of the subsampled of calibration beads
    """
    output = deepcopy(original_beads) # ? First we deepcopy the original beads data, to avoid modifying it by mistake
    selected_beads = []
    """
    Let's modify our output until we reach the desired number of beads.
    You want n beads sampled across both plates A & B (remember that plates C & D are just A & B rotated):
    1. So you're gonna sample n_beads // 2 beads on plate A, and n_beads - (n_beads // 2) beads on plate B.
    2. Add all these beads' names in the selected_beads list.
    3. Then for each A & B bead, you want to add it's counterpart of plate C & D.
    The selection step is already done :)
    """
    A = original_beads['2d']['bead'][original_beads['2d']['bead'].str.contains("A")]
    A = random.sample(A.tolist(), n_beads // 2)

    B = original_beads['2d']['bead'][original_beads['2d']['bead'].str.contains("B")]
    B = random.sample(B.tolist(), n_beads - (n_beads // 2))

    C = deepcopy(A)
    D = deepcopy(B)
    for i in range(len(C)):
        C[i] = C[i].replace("A", "C")
    for i in range(len(D)):
        D[i] = D[i].replace("B", "D")

    selected_beads.extend(A)
    selected_beads.extend(B)
    selected_beads.extend(C)
    selected_beads.extend(D)
    output['2d'] = output['2d'][output['2d']['bead'].isin(selected_beads)].sort_values(by = 'bead', ignore_index = True)
    output['3d'] = output['3d'][output['3d']['bead'].isin(selected_beads)].sort_values(by = 'bead', ignore_index = True)
    return output

sample_n_beads(beads_data, 10)

{'2d':            view   bead  x_2d  y_2d
 0   Beads2D_PA0  A_1_1  -373   858
 1   Beads2D_PA0  A_1_2  -373   784
 2   Beads2D_PA0  A_2_1  -274   858
 3   Beads2D_PA0  A_3_6     1  -178
 4   Beads2D_PA0  A_4_2   264   786
 5   Beads2D_PA0  B_2_1  -179   689
 6   Beads2D_PA0  B_3_4     0  -378
 7   Beads2D_PA0  B_4_2   178   376
 8   Beads2D_PA0  B_4_4   178  -378
 9   Beads2D_PA0  B_5_4   324  -378
 10  Beads2D_LAT  C_1_1  -374   858
 11  Beads2D_LAT  C_1_2  -373   785
 12  Beads2D_LAT  C_2_1  -274   859
 13  Beads2D_LAT  C_3_6     1  -178
 14  Beads2D_LAT  C_4_2   263   787
 15  Beads2D_LAT  D_2_1  -178   689
 16  Beads2D_LAT  D_3_4     0  -378
 17  Beads2D_LAT  D_4_2   177   377
 18  Beads2D_LAT  D_4_4   177  -377
 19  Beads2D_LAT  D_5_4   325  -378,
 '3d':              view   bead      x_3d      y_3d     z_3d
 0   Calib_Beads3D  A_1_1 -602.6390   95.5440  213.567
 1   Calib_Beads3D  A_1_2 -602.6580   95.3980  194.395
 2   Calib_Beads3D  A_2_1 -602.5950   70.1790  213.700
 3   Calib_

In [107]:
# Let's increase the number of calibration beads gradually
spine = lab_1.structures.Spine(os.path.join(root_directory,'Vertebrae2D.mat')) # Initialize the vertebrae structure

# Create a dictionnary to store the results. Access a dictionnary key with brackets, for instance rms_measurements['x']
rms_measurements = {
        'x' : [],
        'y' : [],
        'z' : [],
        '3d' : [],
        'n_beads':[]
    }

"""
PSEUDOCODE
for n_beads in [1...N], do:
    computed_points_3d <- list
    beads_subset <- sample_n_beads(beads_data, n_beads)
    A_view_0 <- get_a(view_0, beads_subset)
    l_view_0 <- dlt(A_view_0)
    A_view_1 <- get_a(view_1, beads_subset)
    l_view_1 <- dlt(A_view_1)
    for each vertebra in vertebrae:
        for each anatomical_landmark in vertebra_points:
        x_point, y_point, z_point <- calculate_3d_point(anatomical_landmark, l_view0, l_view1)
        Append (x_point, y_point, z_point) to computed_points_3d
    for each axis in ['x', ..., '3d']:
        Append rms(SPINE_3D_POINTS, computed_points_3d) to rms_measurements[axis]
    Append n_beads to rms_measurements['n_beads']
"""
for n_beads in range(10, 0, -1):
    computed_points_3d = []
    beads_subset = sample_n_beads(beads_data, n_beads)
    A_view_0 = get_a('Beads2D_'+view_0, beads_subset)
    l_view_0 = lab_1.solver.dlt(A_view_0)
    A_view_1 = get_a('Beads2D_'+view_1, beads_subset)
    l_view_1 = lab_1.solver.dlt(A_view_1)
    for i, v_name in enumerate(spine.vertebrae):

        vert = spine.get(v_name).data # Returns the vertebra data

        for datum in vert.iterrows():
            data = datum[1]
            u0 = data["x_Vertebrae_LAT"]
            v0 = data["y_Vertebrae_LAT"]
            u1 = data["x_Vertebrae_PA0"]
            v1 = data["y_Vertebrae_PA0"]
            point = calculate_3d_point(u0, v0, u1, v1, l_view_0, l_view_1)
            computed_points_3d.append(point)
    computed_points_3d = np.array(computed_points_3d)
    x_rms = rms(SPINE_3D_POINTS[:,0], computed_points_3d[:,0])
    y_rms = rms(SPINE_3D_POINTS[:,1], computed_points_3d[:,1])
    z_rms = rms(SPINE_3D_POINTS[:,2], computed_points_3d[:,2])

    three_d = rms(SPINE_3D_POINTS, computed_points_3d)
    rms_measurements['x'].append(x_rms)
    rms_measurements['y'].append(y_rms)
    rms_measurements['z'].append(z_rms)
    rms_measurements['3d'].append(three_d)
    rms_measurements['n_beads'].append(n_beads)


# Plotly interlude

In [108]:
"""
Here is a basic plotly example. Use it to plot your RMS values !
"""
fig = go.Figure()
fig.update_layout(title = 'Various functions plot',
                  yaxis_title = 'f(x)',
                  xaxis_title = 'x')
x_values = np.linspace(0, 10, 1001)
y_values_0 = np.sin(x_values) * np.sin(2*x_values)
y_values_1 = np.cos(x_values) * np.sin(0.5*np.power(x_values, 2))
fig.add_trace(go.Scatter(
    x = x_values,
    y = y_values_0,
    name = 'sin(x) * sin(2x)'
))
fig.add_trace(go.Scatter(
    x = x_values,
    y = y_values_1,
    name = 'cos(x) * sin(0.5 * x^2))'
))
fig.show()

In [109]:
# TODO : On the same graph, plot all your RMS values wrt. the number of used calibration beads !
fig = go.Figure()
fig.update_layout(title = 'RMS Error with varying number of calibration beads',
                  yaxis_title = 'RMS Error',
                  xaxis_title = 'Number of calibration beads')

fig.add_trace(go.Scatter(
    x = rms_measurements['n_beads'],
    y = rms_measurements['x'],
    name = 'X Axis'
))
fig.add_trace(go.Scatter(
    x = rms_measurements['n_beads'],
    y = rms_measurements['y'],
    name = 'Y axis'
))
fig.add_trace(go.Scatter(
    x = rms_measurements['n_beads'],
    y = rms_measurements['z'],
    name = 'Z axis'
))
fig.add_trace(go.Scatter(
    x = rms_measurements['n_beads'],
    y = rms_measurements['3d'],
    name = '3D Model'
))
fig.show()

# Question 2.2

Here are 4 sets of 8 calibration beads. For each set, determine which one is which (same plate, small volume, medium volume, large volume) and plot calibration error curves.  
Don't hesitate to use the function `plot_selected_beads` from `lab_1.render`to vizualize the selected beads to sort out the subsets !

In [110]:
set_0 = ['A_1_6', 'A_4_6', 'A_1_2', 'A_4_2', 'B_1_5', 'B_5_5', 'B_1_1', 'B_5_1', 'C_1_6', 'C_4_6', 'C_1_2', 'C_4_2', 'D_1_5', 'D_5_5', 'D_1_1', 'D_5_1']
set_1 = ['B_1_1', 'B_2_2', 'B_4_2', 'B_5_3', 'B_1_4', 'B_2_5', 'B_4_3', 'B_5_5', 'D_1_1', 'D_2_2', 'D_4_2', 'D_5_3', 'D_1_4', 'D_2_5', 'D_4_3', 'D_5_5']
set_2 = ['A_1_4', 'A_2_4', 'A_1_5', 'A_2_5', 'B_1_3', 'B_2_3', 'B_1_4', 'B_2_4', 'C_1_4', 'C_2_4', 'C_1_5', 'C_2_5', 'D_1_3', 'D_2_3', 'D_1_4', 'D_2_4']
set_3 = ['A_1_4', 'A_3_4', 'A_1_5', 'A_3_5', 'B_1_3', 'B_3_3', 'B_1_4', 'B_3_4', 'C_1_4', 'C_3_4', 'C_1_5', 'C_3_5', 'D_1_3', 'D_3_3', 'D_1_4', 'D_3_4']

rms_measurements = {
        'x' : [],
        'y' : [],
        'z' : [],
        '3d' : [],
        'plate':[]
    }

large = beads_data["2d"][beads_data['2d']['bead'].isin(set_0)]
same_plate = beads_data["2d"][beads_data['2d']['bead'].isin(set_1)]
small = beads_data["2d"][beads_data['2d']['bead'].isin(set_2)]
medium = beads_data["2d"][beads_data['2d']['bead'].isin(set_3)]

large = {"2d":large}
same_plate = {"2d":same_plate}
small = {"2d":small}
medium = {"2d":medium}

three_large = beads_data["3d"][beads_data['3d']['bead'].isin(set_0)]
three_same_plate = beads_data["3d"][beads_data['3d']['bead'].isin(set_1)]
three_small = beads_data["3d"][beads_data['3d']['bead'].isin(set_2)]
three_medium = beads_data["3d"][beads_data['3d']['bead'].isin(set_3)]

large["3d"] = three_large
same_plate["3d"] = three_same_plate
small["3d"] = three_small
medium["3d"] = three_medium

names = ["same", "small", "medium", "large"]
plates = [same_plate, small, medium, large]

for j, plate in enumerate(plates):
    computed_points_3d = []
    beads_subset = plate
    A_view_0 = get_a('Beads2D_'+view_0, beads_subset)
    l_view_0 = lab_1.solver.dlt(A_view_0)
    A_view_1 = get_a('Beads2D_'+view_1, beads_subset)
    l_view_1 = lab_1.solver.dlt(A_view_1)
    for i, v_name in enumerate(spine.vertebrae):

        vert = spine.get(v_name).data # Returns the vertebra data

        for datum in vert.iterrows():
            data = datum[1]
            u0 = data["x_Vertebrae_LAT"]
            v0 = data["y_Vertebrae_LAT"]
            u1 = data["x_Vertebrae_PA0"]
            v1 = data["y_Vertebrae_PA0"]
            point = calculate_3d_point(u0, v0, u1, v1, l_view_0, l_view_1)
            computed_points_3d.append(point)
    computed_points_3d = np.array(computed_points_3d)
    x_rms = rms(SPINE_3D_POINTS[:,0], computed_points_3d[:,0])
    y_rms = rms(SPINE_3D_POINTS[:,1], computed_points_3d[:,1])
    z_rms = rms(SPINE_3D_POINTS[:,2], computed_points_3d[:,2])

    three_d = rms(SPINE_3D_POINTS, computed_points_3d)
    rms_measurements['x'].append(x_rms)
    rms_measurements['y'].append(y_rms)
    rms_measurements['z'].append(z_rms)
    rms_measurements['3d'].append(three_d)
    rms_measurements['plate'].append(names[j])

In [111]:
fig = go.Figure()
fig.update_layout(title = 'RMS Error with varying calibration size',
                  yaxis_title = 'RMS Error',
                  xaxis_title = 'Bead Size')

fig.add_trace(go.Scatter(
    x = rms_measurements['plate'],
    y = rms_measurements['x'],
    name = 'X Axis'
))
fig.add_trace(go.Scatter(
    x = rms_measurements['plate'],
    y = rms_measurements['y'],
    name = 'Y axis'
))
fig.add_trace(go.Scatter(
    x = rms_measurements['plate'],
    y = rms_measurements['z'],
    name = 'Z axis'
))
fig.add_trace(go.Scatter(
    x = rms_measurements['plate'],
    y = rms_measurements['3d'],
    name = '3D Model'
))
fig.show()

TODO : same thing as before, but now you have to select the provided beads ! Look at the function `sample_n_beads` from Q2.1 if you need some help on how to do this.

In [112]:
def sample_8_beads(original_beads:dict, n_beads:int) -> dict:
    """
    Samples n_beads beads from the original beads set

    Parameters
    ----------
    original_beads : dict
        Dictionnary containing information on 2D and 3D position of the whole set of calibration beads
    n_beads : int
        Number of beads to select

    Returns
    -------
    dict
        Dictionnary containing information on 2D and 3D position of the subsampled of calibration beads
    """
    output = deepcopy(original_beads) # ? First we deepcopy the original beads data, to avoid modifying it by mistake
    selected_beads = []
    """
    Let's modify our output until we reach the desired number of beads.
    You want n beads sampled across both plates A & B (remember that plates C & D are just A & B rotated):
    1. So you're gonna sample n_beads // 2 beads on plate A, and n_beads - (n_beads // 2) beads on plate B.
    2. Add all these beads' names in the selected_beads list.
    3. Then for each A & B bead, you want to add it's counterpart of plate C & D.
    The selection step is already done :)
    """
    A = original_beads['2d']['bead'][original_beads['2d']['bead'].str.contains("A")]
    A = random.sample(A.tolist(), n_beads // 2)

    B = original_beads['2d']['bead'][original_beads['2d']['bead'].str.contains("B")]
    B = random.sample(B.tolist(), n_beads - (n_beads // 2))

    C = deepcopy(A)
    D = deepcopy(B)
    for i in range(len(C)):
        C[i] = C[i].replace("A", "C")
    for i in range(len(D)):
        D[i] = D[i].replace("B", "D")

    selected_beads.extend(A)
    selected_beads.extend(B)
    selected_beads.extend(C)
    selected_beads.extend(D)
    output['2d'] = output['2d'][output['2d']['bead'].isin(selected_beads)].sort_values(by = 'bead', ignore_index = True)
    output['3d'] = output['3d'][output['3d']['bead'].isin(selected_beads)].sort_values(by = 'bead', ignore_index = True)
    return output


# Question 2.3

Using the **small** calibration volume from previous question, plot reconstruction error with respect to the center of gravity of the calibration volume !

In [113]:
rms_measurements = {
    'rms' : [],
    'distance' : []
}

computed_points_3d = []
beads_subset = small
A_view_0 = get_a('Beads2D_'+view_0, beads_subset)
l_view_0 = lab_1.solver.dlt(A_view_0)
A_view_1 = get_a('Beads2D_'+view_1, beads_subset)
l_view_1 = lab_1.solver.dlt(A_view_1)

center_mass = [beads_subset['3d']['x_3d'].mean(), beads_subset['3d']['y_3d'].mean(),beads_subset['3d']['z_3d'].mean()]

for i, v_name in enumerate(spine.vertebrae):

    vert = spine.get(v_name).data # Returns the vertebra data

    for datum in vert.iterrows():
        data = datum[1]
        u0 = data["x_Vertebrae_LAT"]
        v0 = data["y_Vertebrae_LAT"]
        u1 = data["x_Vertebrae_PA0"]
        v1 = data["y_Vertebrae_PA0"]
        point = calculate_3d_point(u0, v0, u1, v1, l_view_0, l_view_1)
        computed_points_3d.append(point)
computed_points_3d = np.array(computed_points_3d)
for i in range(len(computed_points_3d)):

    three_d = rms(SPINE_3D_POINTS[i,:], computed_points_3d[i,:])
    distance = np.linalg.norm(SPINE_3D_POINTS[i,:] - center_mass)

    rms_measurements['rms'].append(three_d)
    rms_measurements['distance'].append(distance)

In [114]:
rms_measurements = pd.DataFrame(rms_measurements)
rms_measurements = rms_measurements.sort_values(by='distance', ascending=True)
fig = go.Figure()
fig.update_layout(title = 'RMS Error with varying distance from center of mass to prediction',
                  yaxis_title = 'RMS Error',
                  xaxis_title = 'Distance from center of mass')

fig.add_trace(go.Scatter(
    x = rms_measurements['distance'],
    y = rms_measurements['rms'],
    name = '3D Model'
))
fig.show()

# Question 3.1 

Here you have to code the function that allow us to add noise to the calibration beads coordinates. Hint : you should take a look at the documentation of `np.random.normal` :)

In [115]:
def add_noise_2d(original_beads:dict, std:float) -> dict:
    """
    Adds noise to the 2D coordinates of the beads

    Parameters
    ----------
    original_beads : dict
        Dictionnary containing information on 2D and 3D position of the a set of calibration beads
    std : float
        Standard deviation for the noise added.

    Returns
    -------
    dict
        Noised beads set
    """
    output = deepcopy(original_beads) # ? First we deepcopy the original beads data, to avoid modifying it by mistake
    # TODO : select 2D beads and add noise to them
    output['2d']['x_2d'] += np.random.normal(0, std, output['2d']['x_2d'].shape)
    output['2d']['y_2d'] += np.random.normal(0, std, output['2d']['y_2d'].shape)

    return output

def add_noise_3d(original_beads:dict, std:float) -> dict:
    """
    Adds noise to the 3D coordinates of the beads

    Parameters
    ----------
    original_beads : dict
        Dictionnary containing information on 2D and 3D position of the a set of calibration beads
    std : float
        Standard deviation for the noise added.

    Returns
    -------
    dict
        Noised beads set
    """
    output = deepcopy(original_beads) # ? First we deepcopy the original beads data, to avoid modifying it by mistake
    # TODO : select 3D beads and add noise to them
    output['3d']['x_3d'] += np.random.normal(0, std, output['3d']['x_3d'].shape)
    output['3d']['y_3d'] += np.random.normal(0, std, output['3d']['y_3d'].shape)
    output['3d']['z_3d'] += np.random.normal(0, std, output['3d']['z_3d'].shape)

    return output


In [116]:
# Let's increase the number of calibration beads gradually
spine = lab_1.structures.Spine(os.path.join(root_directory,'Vertebrae2D.mat')) # Initialize the vertebrae structure

# Create a dictionnary to store the results. Access a dictionnary key with brackets, for instance rms_measurements['x']
rms_measurements = {
        'x' : [],
        'y' : [],
        'z' : [],
        '3d' : [],
        'std':[]
    }

"""
PSEUDOCODE
for n_beads in [1...N], do:
    computed_points_3d <- list
    beads_subset <- sample_n_beads(beads_data, n_beads)
    A_view_0 <- get_a(view_0, beads_subset)
    l_view_0 <- dlt(A_view_0)
    A_view_1 <- get_a(view_1, beads_subset)
    l_view_1 <- dlt(A_view_1)
    for each vertebra in vertebrae:
        for each anatomical_landmark in vertebra_points:
        x_point, y_point, z_point <- calculate_3d_point(anatomical_landmark, l_view0, l_view1)
        Append (x_point, y_point, z_point) to computed_points_3d
    for each axis in ['x', ..., '3d']:
        Append rms(SPINE_3D_POINTS, computed_points_3d) to rms_measurements[axis]
    Append n_beads to rms_measurements['n_beads']
"""

for i in range(0, 110, 10):
    std = i / 100
    computed_points_3d = []
    noised_data = add_noise_2d(beads_data, std)
    #noised_data = add_noise_3d(noised_data, std)

    A_view_0 = get_a('Beads2D_'+view_0, noised_data)
    l_view_0 = lab_1.solver.dlt(A_view_0)
    A_view_1 = get_a('Beads2D_'+view_1, noised_data)
    l_view_1 = lab_1.solver.dlt(A_view_1)

    for i, v_name in enumerate(spine.vertebrae):

        vert = spine.get(v_name).data # Returns the vertebra data

        for datum in vert.iterrows():
            data = datum[1]
            u0 = data["x_Vertebrae_LAT"]
            v0 = data["y_Vertebrae_LAT"]
            u1 = data["x_Vertebrae_PA0"]
            v1 = data["y_Vertebrae_PA0"]
            point = calculate_3d_point(u0, v0, u1, v1, l_view_0, l_view_1)
            computed_points_3d.append(point)
    computed_points_3d = np.array(computed_points_3d)
    x_rms = rms(SPINE_3D_POINTS[:,0], computed_points_3d[:,0])
    y_rms = rms(SPINE_3D_POINTS[:,1], computed_points_3d[:,1])
    z_rms = rms(SPINE_3D_POINTS[:,2], computed_points_3d[:,2])

    three_d = rms(SPINE_3D_POINTS, computed_points_3d)
    rms_measurements['x'].append(x_rms)
    rms_measurements['y'].append(y_rms)
    rms_measurements['z'].append(z_rms)
    rms_measurements['3d'].append(three_d)
    rms_measurements['std'].append(std)

In [117]:
fig = go.Figure()
fig.update_layout(title = 'RMS Error with varying 2D noise',
                  yaxis_title = 'RMS Error',
                  xaxis_title = 'STD of Gaussian Noise')

fig.add_trace(go.Scatter(
    x = rms_measurements['std'],
    y = rms_measurements['x'],
    name = 'X Axis'
))
fig.add_trace(go.Scatter(
    x = rms_measurements['std'],
    y = rms_measurements['y'],
    name = 'Y axis'
))
fig.add_trace(go.Scatter(
    x = rms_measurements['std'],
    y = rms_measurements['z'],
    name = 'Z axis'
))
fig.add_trace(go.Scatter(
    x = rms_measurements['std'],
    y = rms_measurements['3d'],
    name = '3D Model'
))
fig.show()

# Question 3.2

In [118]:
beads_subset = sample_n_beads(beads_data, 8)
# Let's increase the number of calibration beads gradually
spine = lab_1.structures.Spine(os.path.join(root_directory,'Vertebrae2D.mat')) # Initialize the vertebrae structure

# Create a dictionnary to store the results. Access a dictionnary key with brackets, for instance rms_measurements['x']
rms_measurements = {
        'x' : [],
        'y' : [],
        'z' : [],
        '3d' : [],
        'std':[]
    }

"""
PSEUDOCODE
for n_beads in [1...N], do:
    computed_points_3d <- list
    beads_subset <- sample_n_beads(beads_data, n_beads)
    A_view_0 <- get_a(view_0, beads_subset)
    l_view_0 <- dlt(A_view_0)
    A_view_1 <- get_a(view_1, beads_subset)
    l_view_1 <- dlt(A_view_1)
    for each vertebra in vertebrae:
        for each anatomical_landmark in vertebra_points:
        x_point, y_point, z_point <- calculate_3d_point(anatomical_landmark, l_view0, l_view1)
        Append (x_point, y_point, z_point) to computed_points_3d
    for each axis in ['x', ..., '3d']:
        Append rms(SPINE_3D_POINTS, computed_points_3d) to rms_measurements[axis]
    Append n_beads to rms_measurements['n_beads']
"""

for i in range(0, 110, 10):
    std = i / 100
    computed_points_3d = []
    noised_data = add_noise_2d(beads_subset, std)
    #noised_data = add_noise_3d(noised_data, std)

    A_view_0 = get_a('Beads2D_'+view_0, noised_data)
    l_view_0 = lab_1.solver.dlt(A_view_0)
    A_view_1 = get_a('Beads2D_'+view_1, noised_data)
    l_view_1 = lab_1.solver.dlt(A_view_1)

    for i, v_name in enumerate(spine.vertebrae):

        vert = spine.get(v_name).data # Returns the vertebra data

        for datum in vert.iterrows():
            data = datum[1]
            u0 = data["x_Vertebrae_LAT"]
            v0 = data["y_Vertebrae_LAT"]
            u1 = data["x_Vertebrae_PA0"]
            v1 = data["y_Vertebrae_PA0"]
            point = calculate_3d_point(u0, v0, u1, v1, l_view_0, l_view_1)
            computed_points_3d.append(point)
    computed_points_3d = np.array(computed_points_3d)
    x_rms = rms(SPINE_3D_POINTS[:,0], computed_points_3d[:,0])
    y_rms = rms(SPINE_3D_POINTS[:,1], computed_points_3d[:,1])
    z_rms = rms(SPINE_3D_POINTS[:,2], computed_points_3d[:,2])

    three_d = rms(SPINE_3D_POINTS, computed_points_3d)
    rms_measurements['x'].append(x_rms)
    rms_measurements['y'].append(y_rms)
    rms_measurements['z'].append(z_rms)
    rms_measurements['3d'].append(three_d)
    rms_measurements['std'].append(std)


In [119]:
fig = go.Figure()
fig.update_layout(title = 'RMS Error with varying noise on 8 beads subset',
                  yaxis_title = 'RMS Error',
                  xaxis_title = 'STD of Gaussian Noise')

fig.add_trace(go.Scatter(
    x = rms_measurements['std'],
    y = rms_measurements['x'],
    name = 'X Axis'
))
fig.add_trace(go.Scatter(
    x = rms_measurements['std'],
    y = rms_measurements['y'],
    name = 'Y axis'
))
fig.add_trace(go.Scatter(
    x = rms_measurements['std'],
    y = rms_measurements['z'],
    name = 'Z axis'
))
fig.add_trace(go.Scatter(
    x = rms_measurements['std'],
    y = rms_measurements['3d'],
    name = '3D Model'
))
fig.show()

# Question 4.1

In [120]:
# Let's increase the number of calibration beads gradually
spine = lab_1.structures.Spine(os.path.join(root_directory, 'Vertebrae2D.mat'))  # Initialize the vertebrae structure

# Create a dictionnary to store the results. Access a dictionnary key with brackets, for instance rms_measurements['x']
rms_measurements = {
    'x': [],
    'y': [],
    'z': [],
    '3d': [],
    'std': []
}

"""
PSEUDOCODE
for n_beads in [1...N], do:
    computed_points_3d <- list
    beads_subset <- sample_n_beads(beads_data, n_beads)
    A_view_0 <- get_a(view_0, beads_subset)
    l_view_0 <- dlt(A_view_0)
    A_view_1 <- get_a(view_1, beads_subset)
    l_view_1 <- dlt(A_view_1)
    for each vertebra in vertebrae:
        for each anatomical_landmark in vertebra_points:
        x_point, y_point, z_point <- calculate_3d_point(anatomical_landmark, l_view0, l_view1)
        Append (x_point, y_point, z_point) to computed_points_3d
    for each axis in ['x', ..., '3d']:
        Append rms(SPINE_3D_POINTS, computed_points_3d) to rms_measurements[axis]
    Append n_beads to rms_measurements['n_beads']
"""

for i in range(0, 110, 10):
    std = i / 100
    computed_points_3d = []
    #noised_data = add_noise_2d(beads_data, std)
    noised_data = add_noise_3d(noised_data, std)

    A_view_0 = get_a('Beads2D_' + view_0, noised_data)
    l_view_0 = lab_1.solver.dlt(A_view_0)
    A_view_1 = get_a('Beads2D_' + view_1, noised_data)
    l_view_1 = lab_1.solver.dlt(A_view_1)

    for i, v_name in enumerate(spine.vertebrae):

        vert = spine.get(v_name).data  # Returns the vertebra data

        for datum in vert.iterrows():
            data = datum[1]
            u0 = data["x_Vertebrae_LAT"]
            v0 = data["y_Vertebrae_LAT"]
            u1 = data["x_Vertebrae_PA0"]
            v1 = data["y_Vertebrae_PA0"]
            point = calculate_3d_point(u0, v0, u1, v1, l_view_0, l_view_1)
            computed_points_3d.append(point)
    computed_points_3d = np.array(computed_points_3d)
    x_rms = rms(SPINE_3D_POINTS[:, 0], computed_points_3d[:, 0])
    y_rms = rms(SPINE_3D_POINTS[:, 1], computed_points_3d[:, 1])
    z_rms = rms(SPINE_3D_POINTS[:, 2], computed_points_3d[:, 2])

    three_d = rms(SPINE_3D_POINTS, computed_points_3d)
    rms_measurements['x'].append(x_rms)
    rms_measurements['y'].append(y_rms)
    rms_measurements['z'].append(z_rms)
    rms_measurements['3d'].append(three_d)
    rms_measurements['std'].append(std)
fig = go.Figure()
fig.update_layout(title='RMS Error with varying 3D noise',
                  yaxis_title='RMS Error',
                  xaxis_title='STD of Gaussian Noise')

fig.add_trace(go.Scatter(
    x=rms_measurements['std'],
    y=rms_measurements['x'],
    name='X Axis'
))
fig.add_trace(go.Scatter(
    x=rms_measurements['std'],
    y=rms_measurements['y'],
    name='Y axis'
))
fig.add_trace(go.Scatter(
    x=rms_measurements['std'],
    y=rms_measurements['z'],
    name='Z axis'
))
fig.add_trace(go.Scatter(
    x=rms_measurements['std'],
    y=rms_measurements['3d'],
    name='3D Model'
))
fig.show()

# Question 4.2

This question is a bit tricky, as you have to get measurements for couples of values of standard deviation for 2D and 3D noise. A line plot is not very adapted for this, but we could use a heatmap ! Here is a small example :

In [121]:
beads_subset = sample_n_beads(beads_data, 8)
# Let's increase the number of calibration beads gradually
spine = lab_1.structures.Spine(os.path.join(root_directory,'Vertebrae2D.mat')) # Initialize the vertebrae structure

# Create a dictionnary to store the results. Access a dictionnary key with brackets, for instance rms_measurements['x']
rms_measurements = {
        'x' : [],
        'y' : [],
        'z' : [],
        '3d' : []
    }

"""
PSEUDOCODE
for n_beads in [1...N], do:
    computed_points_3d <- list
    beads_subset <- sample_n_beads(beads_data, n_beads)
    A_view_0 <- get_a(view_0, beads_subset)
    l_view_0 <- dlt(A_view_0)
    A_view_1 <- get_a(view_1, beads_subset)
    l_view_1 <- dlt(A_view_1)
    for each vertebra in vertebrae:
        for each anatomical_landmark in vertebra_points:
        x_point, y_point, z_point <- calculate_3d_point(anatomical_landmark, l_view0, l_view1)
        Append (x_point, y_point, z_point) to computed_points_3d
    for each axis in ['x', ..., '3d']:
        Append rms(SPINE_3D_POINTS, computed_points_3d) to rms_measurements[axis]
    Append n_beads to rms_measurements['n_beads']
"""

for i in range(0, 110, 10):
    over_x = []
    over_y = []
    over_z = []
    over_3d = []
    std_2d = i / 100
    for j in range(0, 110, 10):


        std_3d = j / 100
        computed_points_3d = []
        noised_data = add_noise_2d(beads_subset, std_2d)
        noised_data = add_noise_3d(noised_data, std_3d)

        A_view_0 = get_a('Beads2D_'+view_0, noised_data)
        l_view_0 = lab_1.solver.dlt(A_view_0)
        A_view_1 = get_a('Beads2D_'+view_1, noised_data)
        l_view_1 = lab_1.solver.dlt(A_view_1)

        for i, v_name in enumerate(spine.vertebrae):

            vert = spine.get(v_name).data # Returns the vertebra data

            for datum in vert.iterrows():
                data = datum[1]
                u0 = data["x_Vertebrae_LAT"]
                v0 = data["y_Vertebrae_LAT"]
                u1 = data["x_Vertebrae_PA0"]
                v1 = data["y_Vertebrae_PA0"]
                point = calculate_3d_point(u0, v0, u1, v1, l_view_0, l_view_1)
                computed_points_3d.append(point)
        computed_points_3d = np.array(computed_points_3d)
        x_rms = rms(SPINE_3D_POINTS[:,0], computed_points_3d[:,0])
        y_rms = rms(SPINE_3D_POINTS[:,1], computed_points_3d[:,1])
        z_rms = rms(SPINE_3D_POINTS[:,2], computed_points_3d[:,2])

        three_d = rms(SPINE_3D_POINTS, computed_points_3d)

        over_x.append(x_rms)
        over_y.append(y_rms)
        over_z.append(z_rms)
        over_3d.append(three_d)
    rms_measurements['x'].append(over_x)
    rms_measurements['y'].append(over_y)
    rms_measurements['z'].append(over_z)
    rms_measurements['3d'].append(over_3d)


In [122]:
fig = go.Figure()
heatmap = np.array(rms_measurements['3d'])
fig.add_trace(
    go.Heatmap(
        z = heatmap,
        x = np.linspace(0., 10., 11),
        y = np.linspace(0., 10., 11),
        text = heatmap.round(2),
        texttemplate="%{text}",
        colorscale='turbo'
    ))
fig.update_yaxes(title_text='2D Labelling Noise')
fig.update_xaxes(title_text='3D Labelling Noise')
fig.update_layout(title = '3D Reconstruction error as noise in 2D and 3D vary')
fig.update_traces(showscale=False)
fig.show()

In [123]:
fig = go.Figure()
heatmap = np.array(rms_measurements['x'])
fig.add_trace(
    go.Heatmap(
        z = heatmap,
        x = np.linspace(0., 10., 11),
        y = np.linspace(0., 10., 11),
        text = heatmap.round(2),
        texttemplate="%{text}",
        colorscale='turbo'
    ))
fig.update_yaxes(title_text='2D Labelling Noise')
fig.update_xaxes(title_text='3D Labelling Noise')
fig.update_layout(title = 'x Axis Reconstruction error as noise in 2D and 3D vary')
fig.update_traces(showscale=False)
fig.show()

In [124]:
fig = go.Figure()
heatmap = np.array(rms_measurements['y'])
fig.add_trace(
    go.Heatmap(
        z = heatmap,
        x = np.linspace(0., 10., 11),
        y = np.linspace(0., 10., 11),
        text = heatmap.round(2),
        texttemplate="%{text}",
        colorscale='turbo'
    ))
fig.update_yaxes(title_text='2D Labelling Noise')
fig.update_xaxes(title_text='3D Labelling Noise')
fig.update_layout(title = 'y Axis Reconstruction error as noise in 2D and 3D vary')
fig.update_traces(showscale=False)
fig.show()

In [125]:
fig = go.Figure()
heatmap = np.array(rms_measurements['z'])
fig.add_trace(
    go.Heatmap(
        z = heatmap,
        x = np.linspace(0., 10., 11),
        y = np.linspace(0., 10., 11),
        text = heatmap.round(2),
        texttemplate="%{text}",
        colorscale='turbo'
    ))
fig.update_yaxes(title_text='2D Labelling Noise')
fig.update_xaxes(title_text='3D Labelling Noise')
fig.update_layout(title = 'z Axis Reconstruction error as noise in 2D and 3D vary')
fig.update_traces(showscale=False)
fig.show()